In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# все любят картиночки:)

In [2]:
# docker контейнер уже должен содержать в себе скачанные данные
# скачаны отсюда: https://grouplens.org/datasets/movielens/
DATA_DIR = "/data/ml-latest"

In [3]:
# смотрим, какие файлы у нас есть
!ls {DATA_DIR}

README.txt	   genome-tags.csv  movies.csv	 tags.csv     tf_idf.parquet
genome-scores.csv  links.csv	    ratings.csv  tf_idf.json


In [4]:
# обратим внимание на файл с оценками
!head {DATA_DIR}/ratings.csv

In [5]:
# для подготовки данных будем использвоать Apache Spark
# (популярный фреймворк распределённых вычислений)
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    # если виртуальной машине нельзя добавить памяти, можно использовать меньше
    .config("spark.driver.memory", "8g")
    # можно явно количество ядер, которые будет использовать Spark
    # либо поставить звёздочку для всех доступных виртуальной машине
    .master("local[*]")
    .getOrCreate()
)

In [6]:
# считываем данные из CSV
# и преобразуем время проставления оценки из целого числа в дату со временем
import os
import pyspark.sql.functions as sql_func

ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, "ratings.csv"),
        header=True,
        inferSchema=True
    )
    # если используется меньше памяти,
    # то здесь можно взять не все данные, а а небольшую выборку
    # даже при fraction=0.01 качественная картина не меняется
    .sample(withReplacement=False, fraction=0.1, seed=0)
    .withColumn("rating_datetime", sql_func.from_unixtime("timestamp"))
    .drop("timestamp")
    .cache()
)

In [7]:
import os
import pyspark.sql.functions as sql_func

# поскольку в Parquet схема данных хранится внутри самого файла, читать их очень просто
tf_idf = spark.read.parquet(os.path.join(DATA_DIR, "tf_idf.parquet")).cache()
tf_idf.show()

+-------+--------------------+--------------------+
|movieId|               title|              tf_idf|
+-------+--------------------+--------------------+
|     35|   Carrington (1995)|(1024,[8,74,189,2...|
|    503| New Age, The (1994)|(1024,[434,769,82...|
|    583|Dear Diary (Caro ...|(1024,[434,741,84...|
|    594|Snow White and th...|(1024,[29,52,60,8...|
|    610|  Heavy Metal (1981)|(1024,[32,93,112,...|
|    614|       Loaded (1994)|(1024,[263,434],[...|
|    761| Phantom, The (1996)|(1024,[43,169,196...|
|    880|Island of Dr. Mor...|(1024,[44,81,219,...|
|   1369|I Can't Sleep (J'...|(1024,[263,434],[...|
|   1519|Broken English (1...|(1024,[434,829],[...|
|   1589|     Cop Land (1997)|(1024,[37,61,164,...|
|   1815|         Eden (1997)|(1024,[434,829],[...|
|   1881|Quest for Camelot...|(1024,[57,165,337...|
|   2080|Lady and the Tram...|(1024,[29,37,83,1...|
|   2324|Life Is Beautiful...|(1024,[3,31,32,45...|
|   2444|24 7: Twenty Four...|(1024,[122,221,43...|
|   2445|At 

In [8]:
# оцениваем размеры данных
print("всего пользователей:", ratings.select("userId").distinct().count())
print("всего фильмов:", ratings.select("movieId").distinct().count())
print("всего оценок:", ratings.count())

всего пользователей: 229897
всего фильмов: 26166
всего оценок: 2599745


In [9]:
# достаточно хорошим baseline является предсказывать среднюю оценку
mean_rating = ratings.agg(sql_func.avg("rating")).first()[0]
print("средняя оценка:", mean_rating)

средняя оценка: 3.5280369805500156


In [10]:
# функция, с помощью которой мы будем вычислять RMSE на обучающей выборке
from pyspark.sql import DataFrame
import numpy as np

def simple_evaluate(predictions_df: DataFrame) -> float:
    return np.sqrt(
            ratings
            .join(
                predictions_df,
                ["movieId", "userId"]
            ).select(
                sql_func.pow(
                    ratings.rating - predictions_df.prediction,
                    2
                ).alias("squared_error")
            )
            .agg(sql_func.avg("squared_error"))
            .first()[0]
    )

In [11]:
# рекомендуем любому пользователю любой фильм случайно
mean_predictions = ratings.withColumn("prediction", sql_func.lit(mean_rating))
print("ошибка предсказания:", simple_evaluate(mean_predictions))

ошибка предсказания: 1.065588580528197


In [12]:
# посмотрим на распределение средних оценок разных фильмов
movie_ratings = (
    ratings
    .groupBy("movieId")
    .agg(sql_func.avg("rating").alias("avg_movie_rating"))
    .cache()
)

In [13]:
# у разных пользователей разные распределения оценок
# кто-то более придирчив, а кто-то всем ставит пятёрки
user_ratings = (
    ratings
    .groupBy("userId")
    .agg(sql_func.avg("rating").alias("avg_user_rating"))
    .cache()
)

In [14]:
# а можем и не полусумму, а подобрать коэффициенты
# с помощью линейной регрессии
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

train = (
    VectorAssembler(
        inputCols = ["avg_movie_rating", "avg_user_rating", "tf_idf"],
        outputCol = "features"
    ).transform(
        ratings
        .join(movie_ratings, "movieId")
        .join(user_ratings, "userId")
        .join(tf_idf, "movieId")
    )
    .withColumnRenamed("rating", "label")
    .select("movieId", "userId", "label", "features")
    .cache()
)


In [15]:
train.head()

Row(movieId=91500, userId=1, label=2.5, features=SparseVector(1026, {0: 3.5849, 1: 2.5, 20: 139.6376, 37: 22.5082, 39: 14.5658, 44: 14.0427, 47: 4.7746, 48: 30.618, 63: 55.395, 66: 5.5972, 72: 5.9623, 78: 58.3801, 89: 5.4247, 107: 5.0259, 110: 304.7821, 122: 5.46, 124: 4.3874, 130: 5.2441, 135: 176.1395, 182: 10.1681, 184: 5.7494, 202: 10.4964, 205: 10.92, 210: 6.35, 236: 6.2903, 241: 4.9741, 247: 10.1681, 254: 12.424, 261: 7.9498, 265: 35.8053, 276: 53.0145, 287: 5.5019, 300: 68.2947, 301: 5.016, 305: 123.9866, 307: 5.1272, 323: 5.3214, 327: 11.9246, 329: 28.8846, 330: 4.205, 345: 1.9462, 356: 5.5568, 358: 4.6039, 361: 61.1301, 370: 62.9784, 385: 23.6087, 389: 14.0981, 394: 5.7091, 396: 11.8094, 404: 74.1915, 406: 4.8386, 411: 145.9017, 421: 5.2734, 423: 126.515, 436: 9.1098, 439: 3.3299, 440: 4.8114, 442: 105.4234, 447: 87.1953, 450: 4.1156, 461: 19.7235, 494: 100.439, 497: 135.0977, 507: 8.4395, 520: 5.0798, 531: 10.82, 537: 4.5888, 559: 11.2785, 566: 5.2863, 590: 5.5797, 591: 6.014

In [16]:
linear_model = LinearRegression().fit(train)
stacked_prediction = (
    ratings
    .join(linear_model.transform(train), ["movieId", "userId"])
    .select("movieId", "userId", "prediction")
)
print("ошибка предсказания:", simple_evaluate(stacked_prediction))

ошибка предсказания: 0.8347906878605614


In [17]:
# получаем некоторую формулу для предсказания оценки, которую можно использовать для рекомендаций
print(
    "[на сколько пользователь оценит фильм] = {} + {} * [средняя оценка этого фильма] + {} * [средняя оценка из поставленных этим пользователем]"
    .format(
        round(linear_model.intercept, 2),
        round(linear_model.coefficients[0], 2),
        round(linear_model.coefficients[1], 2)
    )
)

[на сколько пользователь оценит фильм] = -2.44 + 0.83 * [средняя оценка этого фильма] + 0.87 * [средняя оценка из поставленных этим пользователем]


Эта формула является частным случаем рекомендтельной архитектуры, когда мы для заданного пользователя $u$ получаем рекомендации в два этапа:

1. составляем список объектов $i$, которые в принципе могут заинтересовать пользователя $u$
1. ранжируем этот список объектов по некоторому правилу

В частности, правило может основываться на присвоении каждому объекту из списка некоторой релевантности как функции от свойств пользователя и самого объекта: $r\left(u,i\right)$

В простейшем примере, рассмотренном в этой тетради $r\left(u,i\right)=ar_i+br_u+c$ - линейная функция от двух переменных:

1. $r_i$ - одного свойства объекта (популярности фильма в виде средней оценки) и
1. $r_u$ - одного свойства пользователя ("разборчивости" в виде среднего от распределения всех оценок в истории этого пользователя)